In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import os
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
src_path_train = "/kaggle/input/test-dataset/Google_Recaptcha_V2_Images_Dataset/images/"
batch_size = 64

train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        #rotation_range=20,
        zoom_range=0.05,
        #width_shift_range=0.05,
        #height_shift_range=0.05,
        #shear_range=0.05,
        horizontal_flip=True,
        #fill_mode="nearest",
        validation_split=0.20)


train_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='training',
    shuffle=True,
    seed=42
)

valid_generator = train_datagen.flow_from_directory(
    directory=src_path_train,
    target_size=(150, 150),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    subset='validation',
    shuffle=True,
    seed=42
)

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow.keras import applications
from tensorflow.keras.metrics import Accuracy


ROWS = 150
COLS = 150
input_shape = (ROWS, COLS, 3)


base_model = applications.InceptionV3(weights='imagenet', include_top=False,input_shape=(ROWS, COLS,3))

for i in base_model.layers[0:150]:
    i.trainable = False
for i in base_model.layers[150::]:
    i.trainable = True
    
add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))
add_model.add(Dense(12, activation='softmax'))

model = add_model
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])
model.summary()

In [ ]:
from sklearn.utils import compute_class_weight
train_classes = train_generator.classes
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_classes),
                                        y = train_classes                                                    
                                    )
class_weights = dict(zip(np.unique(train_classes), class_weights))

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
checkp = ModelCheckpoint('./bestmodel.tf', monitor = 'val_loss', verbose = 1, save_best_only = True)
reduce = ReduceLROnPlateau(monitor = 'val_loss', patience = 20, verbose = 1)

history = model.fit(train_generator, validation_data=valid_generator, callbacks=[checkp],
                    class_weight=class_weights, 
                    epochs=25)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,10))
plt.subplot(2,1,1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Losses')
plt.legend(['train loss','val loss'])
plt.title('Loss function wrt epochs')

plt.subplot(2,1,2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train acc' , 'val acc'])
plt.title('Model accuracy wrt Epoch')

In [ ]:
from keras.models import load_model
model = load_model('bestModel.tf')
pred = model.predict(X_test)

In [ ]:
pred = np.argmax(pred, axis = 1)
yres = np.argmax(y_test,axis= 1)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
target_name = []
for i in sorted(info) :
    target_name.append(info[i])
print('Accuracy : ' + str(accuracy_score(yres, pred)))
print(classification_report(yres, pred, target_names = target_name))